In [6]:
import pandas as pd
import datetime
import numpy as np
from tqdm import tqdm_notebook
from datetime import datetime,timedelta
import os
from timeit import default_timer as timer
from scipy import stats
from sklearn import linear_model
os.chdir('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA')
file_list=os.listdir('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA')
file_list.sort()

In [7]:
file_list

['.DS_Store',
 '2019.01.02.csv',
 '2019.01.03.csv',
 '2019.01.04.csv',
 '2019.01.07.csv',
 '2019.01.08.csv',
 '2019.01.09.csv',
 '2019.01.10.csv',
 '2019.01.11.csv',
 '2019.01.14.csv',
 '2019.01.15.csv',
 '2019.01.16.csv',
 '2019.01.17.csv',
 '2019.01.18.csv',
 '2019.01.21.csv',
 '2019.01.22.csv',
 '2019.01.23.csv',
 '2019.01.24.csv',
 '2019.01.25.csv',
 '2019.01.28.csv',
 '2019.01.29.csv',
 '2019.01.30.csv',
 '2019.01.31.csv',
 '2019.02.01.csv',
 '2019.02.11.csv',
 '2019.02.12.csv',
 '2019.02.13.csv',
 '2019.02.14.csv',
 '2019.02.15.csv',
 '2019.02.18.csv',
 '2019.02.19.csv',
 '2019.02.20.csv',
 '2019.02.21.csv',
 '2019.02.22.csv',
 '2019.02.25.csv',
 '2019.02.26.csv',
 '2019.02.27.csv',
 '2019.02.28.csv',
 '2019.03.01.csv',
 '2019.03.04.csv',
 '2019.03.05.csv',
 '2019.03.06.csv',
 '2019.03.07.csv',
 '2019.03.08.csv',
 '2019.03.11.csv',
 '2019.03.12.csv',
 '2019.03.13.csv',
 '2019.03.14.csv',
 '2019.03.15.csv',
 '2019.03.18.csv',
 '2019.03.19.csv',
 '2019.03.20.csv',
 '2019.03.21.csv'

In [8]:
class categorise():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[25,50,75]
        
    def fit(self,array):
        positive=array[array>0]
        negative=array[array<0]
        self.threshold.append(np.percentile(negative,self.percentiles))   
        self.threshold.append(np.percentile(positive,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
            if i>=0:
                if i<self.threshold[1][0]:
                    temp.append(5)
                elif i<self.threshold[1][1]:
                    temp.append(6)
                elif i<self.threshold[1][2]:
                    temp.append(7)
                else:
                    temp.append(8)
            if i<0:
                if i>self.threshold[0][2]:
                    temp.append(4)
                elif i>self.threshold[0][1]:
                    temp.append(3)
                elif i>self.threshold[0][0]:
                    temp.append(2)
                else:
                    temp.append(1)
        return np.asarray(temp)
    
class LinearRegression(linear_model.LinearRegression):
    """
    LinearRegression class after sklearn's, but calculate t-statistics
    and p-values for model coefficients (betas).
    Additional attributes available after .fit()
    are `t` and `p` which are of the shape (y.shape[1], X.shape[1])
    which is (n_features, n_coefs)
    This class sets the intercept to 0 by default, since usually we include it
    in X.
    """
    def __init__(self, fit_intercept=True, normalize=False, copy_X=True,
                     n_jobs=1):
            self.fit_intercept = fit_intercept
            self.normalize = normalize
            self.copy_X = copy_X
            self.n_jobs = n_jobs
    def fit(self, X, y, n_jobs=1):
        self = super(LinearRegression, self).fit(X, y, n_jobs)

        sse = np.sum((self.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
        se = np.array([
            np.sqrt(np.diagonal(sse[i] * np.linalg.inv(np.dot(X.T, X))))
                                                    for i in range(sse.shape[0])
                    ])

        self.t = self.coef_ / se
        self.p = 2 * (1 - stats.t.cdf(np.abs(self.t), y.shape[0] - X.shape[1]))
        return self                    

In [25]:
def calc_smart_price(dataset):
    data=dataset[:]
    
    #to combat the limit up event, where price is set to 0. 
    rows=(data.loc[:,'BidPrice1']==0) #count rows of bid price equal 0
    if (np.any(rows)): #if there is such a row
        data.at[rows,'BidPrice1']=data.loc[rows,'AskPrice1'] #for that row, assign ask price to it
    rows=(data.loc[:,'AskPrice1']==0) #do the same for ask price
    if (np.any(rows)):
        data.at[rows,'AskPrice1']=data.loc[rows,'BidPrice1'] 
        
    data['smart_price']=data.loc[:,'BidPrice1']*data.loc[:,'AskVol1']+data.loc[:,'AskPrice1']*data.loc[:,'BidVol1']
    data.at[:,'smart_price']=data.loc[:,'smart_price']/(data.loc[:,['BidVol1','AskVol1']].sum(axis=1))  
    return data

def calc_future_price(dataset,time_ahead=30):
    data=dataset[:]
    future_price=[]
    length=len(data)
    for i in range(len(data)):
        current_time=data[i,44]+timedelta(seconds=time_ahead)
        #print(data[i,44])
        j=0
        #print(current_time)
        while((i+j)<length and current_time>data[(i+j),44]):
            j+=1
        #print(i,j,(data[(i+j-1),44]))
        if (i+j)<length:
            future_price.append(data[(i+j),51]) #51 is the index for smart price            
        else:
            future_price.append(np.nan)
    future_price=np.asarray(future_price)
    future_price=np.expand_dims(future_price,axis=1)
    return np.concatenate((data,future_price),axis=1)


def calc_edge(dataset):
    data=dataset.copy()
    temp=data[:,52]-data[:,51]
    temp=np.expand_dims(temp,axis=1)
    return np.concatenate((data,temp),axis=1)

def set_index(dataset):
    data=dataset[:]
    index=data[:,44]
    new_index=[]
    for j in range(len(index)):
        i=str(index[j]*1000)
        if len(i)==11:
            i='0'+i
        i=i[:-10]+':'+i[-10:]
        i=i[:-8]+':'+i[-8:]
        i=i[:-6]+':'+i[-6:]
        new_index.append(datetime.strptime(i,"%H:%M:%S:%f"))
    data[:,44]=new_index
    return data

def calc_sma_fast(dataset,duration=1): #faster way to calculate SMA, 0.05 seconds for 5000 rows
    data=dataset[:]
    sma_values=[] 
    smart_sum=np.cumsum(data[:,51])
    for i in range(len(data)):
        last_time=data[i,44]-timedelta(minutes=duration)
        j=220*duration#4x60=240
        while(i-j>0 and data[i-j,44]>last_time):
            j+=1
        if (i-j>=0):
            sma=(smart_sum[i]-smart_sum[i-j])/(j)
            sma_values.append(sma)
        else:
            sma=smart_sum[i]/(i+1)
            sma_values.append(sma)

    sma_values=np.asarray(sma_values)
    sma_values=data[:,51]-sma_values
    sma_values=np.expand_dims(sma_values,axis=1)
    return np.concatenate((data,sma_values),axis=1)     


def process(dataset):
    data=dataset[:]
    data=calc_smart_price(data).values #51
    data=set_index(data)
    data=calc_future_price(data) #52
    data=calc_edge(data) #53
    data=calc_sma_fast(data,duration=1) #54
    return data
ma_dict={'-4':'1',
        '-3':'5',
        '-2':'15',
        '-1':'30',
        '54':'1',
        '55':'5',
        '56':'15',
        '57':'30'}    
#ignore below        
'''def calc_vwap(dataset,duration=1): #to be implement
    data=dataset[:]
    for i in data[:,44]:
        last_time=i-timedelta(minutes=duration)
        rolling=data[(data[:,44]>=last_time) & (data[:,44]<i)]
        high=rolling[:,51].max()
        low=rolling[:,51].min()
        avg=(rolling[-1,51]+high+low)/3
def calc_rsi(dataset)        '''     

'def calc_vwap(dataset,duration=1): #to be implement\n    data=dataset[:]\n    for i in data[:,44]:\n        last_time=i-timedelta(minutes=duration)\n        rolling=data[(data[:,44]>=last_time) & (data[:,44]<i)]\n        high=rolling[:,51].max()\n        low=rolling[:,51].min()\n        avg=(rolling[-1,51]+high+low)/3\ndef calc_rsi(dataset)        '

In [26]:
file_list[1][-3:]
df_list=[]
name_list=[]
path='/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/'
for file in file_list: #read all files and add them to file_list
    if file[-3:]=='csv': #check if file is a CSV
        name_list.append(file)
        df_list.append(process(pd.read_csv(path+file)))
        print(file,'read')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


2019.01.02.csv read
2019.01.03.csv read
2019.01.04.csv read
2019.01.07.csv read
2019.01.08.csv read
2019.01.09.csv read
2019.01.10.csv read
2019.01.11.csv read
2019.01.14.csv read
2019.01.15.csv read
2019.01.16.csv read
2019.01.17.csv read
2019.01.18.csv read
2019.01.21.csv read
2019.01.22.csv read
2019.01.23.csv read
2019.01.24.csv read
2019.01.25.csv read
2019.01.28.csv read
2019.01.29.csv read
2019.01.30.csv read
2019.01.31.csv read
2019.02.01.csv read
2019.02.11.csv read
2019.02.12.csv read
2019.02.13.csv read
2019.02.14.csv read
2019.02.15.csv read
2019.02.18.csv read
2019.02.19.csv read
2019.02.20.csv read
2019.02.21.csv read
2019.02.22.csv read
2019.02.25.csv read
2019.02.26.csv read
2019.02.27.csv read
2019.02.28.csv read
2019.03.01.csv read
2019.03.04.csv read
2019.03.05.csv read
2019.03.06.csv read
2019.03.07.csv read
2019.03.08.csv read
2019.03.11.csv read
2019.03.12.csv read
2019.03.13.csv read
2019.03.14.csv read
2019.03.15.csv read
2019.03.18.csv read
2019.03.19.csv read


In [18]:
df_list[0][0]

In [19]:
#run regressions against all 4 moving averages
df_path='/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/'

final_df=pd.DataFrame()    

for i in range(len(df_list)): #for each 20 day rolling window
    if i<19:
        continue
    print(name_list[i])

    #get -19 day
    x=df_list[i-19][:,54]
    #get -18 to 0 day (19 days in total)
    for k in range((i-18),i+1): #get 20 day moving averages
        x=np.concatenate((x,df_list[k][:,54]))

    cat=categorise()
    cat.fit(x) #calculate quartile thresholds for past 20 days

    #get x,y for regression
    x=df_list[i][:,54] #column for SMA
    x=x.astype(float)        
    y=df_list[i][:,53] #column for edge
    y=y.astype(float)

    #removing all NA
    isnum=(~np.isnan(x)) & (~np.isnan(y))
    y=y[isnum]
    x=x[isnum]        

    #get the quartiles of the SMA
    category=cat.return_quartile(x)

    reg_result={}
    reg_result['date']=name_list[i]
    reg_result['total_obs']=len(x)

    for quartile in range(1,9):

        #filter by quartile
        filtered=(category==quartile) 
        q='quartile_'+str(quartile)+'_'
        if (np.any(filtered)):
            new_x=x[filtered]
            new_y=y[filtered] 
            new_x=np.expand_dims(new_x,1)
            new_y=np.expand_dims(new_y,1)
            lin_model=LinearRegression(fit_intercept=False)
            lin_model.fit(new_x,new_y)

            reg_result[(q+'slope')]=lin_model.coef_[0][0]

            reg_result[(q+'p_val')]=lin_model.p[0][0]
            reg_result[(q+'x_mean')]=np.mean(new_x)
            reg_result[(q+'x_std')]=np.std(new_x)
            reg_result[(q+'num_obs')]=len(new_x)

        else:
            reg_result[(q+'slope')]='na'

            reg_result[(q+'p_val')]='na'
            reg_result[(q+'x_mean')]='na'
            reg_result[(q+'x_std')]='na'
            reg_result[(q+'num_obs')]='0'

    final_df=final_df.append(reg_result,ignore_index=True)                
temp=df_path+'result_binary_1day_8split_1minute_ma_nointercept.csv'
final_df.to_csv(temp)
print('done',temp)

2019.01.29.csv
2019.01.30.csv
2019.01.31.csv
2019.02.01.csv
2019.02.11.csv
2019.02.12.csv
2019.02.13.csv
2019.02.14.csv
2019.02.15.csv
2019.02.18.csv
2019.02.19.csv
2019.02.20.csv
2019.02.21.csv
2019.02.22.csv
2019.02.25.csv
2019.02.26.csv
2019.02.27.csv
2019.02.28.csv
2019.03.01.csv
2019.03.04.csv
2019.03.05.csv
2019.03.06.csv
2019.03.07.csv
2019.03.08.csv
2019.03.11.csv
2019.03.12.csv
2019.03.13.csv
2019.03.14.csv
2019.03.15.csv
2019.03.18.csv
2019.03.19.csv
2019.03.20.csv
2019.03.21.csv
2019.03.22.csv
2019.03.25.csv
2019.03.26.csv
2019.03.27.csv
2019.03.28.csv
2019.03.29.csv
2019.04.01.csv
2019.04.02.csv
2019.04.03.csv
2019.04.04.csv
2019.04.08.csv
2019.04.09.csv
2019.04.10.csv
2019.04.11.csv
2019.04.12.csv
2019.04.15.csv
2019.04.16.csv
2019.04.17.csv
2019.04.18.csv
2019.04.19.csv
2019.04.22.csv
2019.04.23.csv
2019.04.24.csv
2019.04.25.csv
2019.04.26.csv
2019.04.29.csv
2019.04.30.csv
2019.05.06.csv
2019.05.07.csv
2019.05.08.csv
2019.05.09.csv
done /Users/hudsonyeo/Desktop/Python/leo

In [29]:
data=pd.read_csv('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/result_binary_1day_8split_1minute_ma_nointercept.csv')
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
for i in range(1,9):
    col='quartile_'+str(i)+'_slope'
    row=data.loc[:,col]
    x=[]
    for j in row:
        if is_number(j) and j==j:
            k=float(j)
            #print(k)
            x.append(k)

            
    x=np.asarray(x)
    pos=x[x>0]
    perc=len(pos)/len(x)
    print('category:',i,'percentage positive:',round(perc,7),'mean slope:',round(row.mean(),7),'std dev',round(row.std(),7))
    
    
    
    

In [33]:
df_list[10][0]

array(['2019-01-16', 1547600399898654, 'TA905', 0, 6044, 6118, 1115984, 0,
       0, 0, 6120, 6118, 6114, 6112, 6110, 6108, 6120, 6122, 6124, 6126,
       6128, 10, 101, 138, 343, 178, 68, 247, 157, 205, 554, 640270,
       19575647220, 1122674, 0, 6116, 6116, 0, 0, 6348, 5740, 32486,
       21886, 20190116, datetime.datetime(1900, 1, 1, 9, 0), 20190116, 0,
       32400000, 20190116, 5932, 6184, 6118.25641025641,
       6120.674418604651, 2.418008348240619, 0.0], dtype=object)

In [27]:
##binary
#run regressions against all 4 moving averages
df_path='/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/'

final_df=pd.DataFrame()    

for i in range(len(df_list)): #for each 20 day rolling window
    if i<19:
        continue
    print(name_list[i])

    #get -19 day
    x=df_list[i-19][:,54]
    #get -18 to 0 day (19 days in total)
    for k in range((i-18),i+1): #get 20 day moving averages
        x=np.concatenate((x,df_list[k][:,54]))

    cat=categorise()
    cat.fit(x) #calculate quartile thresholds for past 20 days

    #get x,y for regression
    x=df_list[i][:,54] #column for SMA
    x=x.astype(float)        
    y=df_list[i][:,53] #column for edge
    y=y.astype(float)

    #removing all NA
    isnum=(~np.isnan(x)) & (~np.isnan(y))
    y=y[isnum]
    x=x[isnum]        

    #get the quartiles of the SMA
    category=cat.return_quartile(x)

    reg_result={}
    reg_result['date']=name_list[i]
    reg_result['total_obs']=len(x)

    for quartile in range(1,9):

        #filter by quartile
        filtered=(category==quartile) 
        q='quartile_'+str(quartile)+'_'
        if (np.any(filtered)):
            new_x=x[filtered]
            new_x[new_x>0]=1
            new_x[new_x<0]=-1
            new_y=y[filtered] 
            new_x=np.expand_dims(new_x,1)
            new_y=np.expand_dims(new_y,1)
            lin_model=LinearRegression(fit_intercept=False)
            lin_model.fit(new_x,new_y)

            reg_result[(q+'slope')]=lin_model.coef_[0][0]

            reg_result[(q+'p_val')]=lin_model.p[0][0]
            reg_result[(q+'x_mean')]=np.mean(new_x)
            reg_result[(q+'x_std')]=np.std(new_x)
            reg_result[(q+'num_obs')]=len(new_x)

        else:
            reg_result[(q+'slope')]='na'

            reg_result[(q+'p_val')]='na'
            reg_result[(q+'x_mean')]='na'
            reg_result[(q+'x_std')]='na'
            reg_result[(q+'num_obs')]='0'

    final_df=final_df.append(reg_result,ignore_index=True)                
temp=df_path+'result_binary_1day_8split_1minute_ma_nointercept.csv'
final_df.to_csv(temp)
print('done',temp)

2019.01.29.csv
2019.01.30.csv
2019.01.31.csv
2019.02.01.csv
2019.02.11.csv
2019.02.12.csv
2019.02.13.csv
2019.02.14.csv
2019.02.15.csv
2019.02.18.csv
2019.02.19.csv
2019.02.20.csv
2019.02.21.csv
2019.02.22.csv
2019.02.25.csv
2019.02.26.csv
2019.02.27.csv
2019.02.28.csv
2019.03.01.csv
2019.03.04.csv
2019.03.05.csv
2019.03.06.csv
2019.03.07.csv
2019.03.08.csv
2019.03.11.csv
2019.03.12.csv
2019.03.13.csv
2019.03.14.csv
2019.03.15.csv
2019.03.18.csv
2019.03.19.csv
2019.03.20.csv
2019.03.21.csv
2019.03.22.csv
2019.03.25.csv
2019.03.26.csv
2019.03.27.csv
2019.03.28.csv
2019.03.29.csv
2019.04.01.csv
2019.04.02.csv
2019.04.03.csv
2019.04.04.csv
2019.04.08.csv
2019.04.09.csv
2019.04.10.csv
2019.04.11.csv
2019.04.12.csv
2019.04.15.csv
2019.04.16.csv
2019.04.17.csv
2019.04.18.csv
2019.04.19.csv
2019.04.22.csv
2019.04.23.csv
2019.04.24.csv
2019.04.25.csv
2019.04.26.csv
2019.04.29.csv
2019.04.30.csv
2019.05.06.csv
2019.05.07.csv
2019.05.08.csv
2019.05.09.csv
done /Users/hudsonyeo/Desktop/Python/leo